In [16]:
using CSV

raw = CSV.read("courses_and_GPA.csv")
(m,n) = size(raw)

courses = raw[:,1]
gpa = convert(Matrix{Float64}, raw[:,2:n])
pre = [[],[],[],[2],[2,3],[3],[3],[69,70,3],[51],[51],[49],[51],[6,7],[55,3],[51,59,3],[70,6],[3,51],[51],[18,3],[69,1],[45,51],[3,69],[5,6],[5,6],[5,6],[1],[3,68],[23],[47,3,5],[4,5],[6,47],[69,6],[5,6],[55,73],[1],[3,69],[70,6],[],[3,51],[],[24],[24],[32],[],[44],[44],[69],[47],[71,73],[69],[69],[58],[52],[47],[69],[47],[],[57],[47],[47],[51],[47],[53],[56,57],[64],[64],[56],[],[68],[],[47],[48],[68]]                                 

add_math = [55,73,8,10,14,15,18,19,46,47,50,51,52,53,54,56,58,60,61,62,12,63,64,65,66,67,48,49,71,72]
theory = [37,16]
soft_hard = [7,13,23,24,30,33,41,42]
app = [8,9,14,15,17,18,22,27,28,29,32,35,]
elect = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,40]
len = length(gpa)
;

In [18]:
# Suppose a student wants to take more applications and additional math courses, 
# as he wants to double major in math/stats:

In [17]:
using JuMP, Cbc

NUM_COURSES = 20

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

@variable(m, x[1:len], Bin)  # whether or not a course is taken

# Must satisfy degree requirements
@constraint(m, sum( x[add_math[i]] for i in 1:length(add_math) ) >= 4)
@constraint(m, sum( x[theory[i]] for i in 1:length(theory) ) >= 1)
@constraint(m, sum( x[soft_hard[i]] for i in 1:length(soft_hard) ) >= 2)
@constraint(m, sum( x[app[i]] for i in 1:length(app) ) >= 4)
@constraint(m, sum( x[elect[i]] for i in 1:length(elect) ) >= 2)

# A class can not count for more than one degree requirement
@constraint(m, sum( x[i] for i in 1:len ) == NUM_COURSES)

# Must satisfy prerequisite requirements
for i in 1:len
    for j in pre[i]
        @constraint(m, x[i] <= x[j])
    end
end

# Maximize GPA
@objective(m, Max, sum(x[i]*gpa[i] for i in 1:len) / NUM_COURSES);

optimize!(m)

opt_gpa = 0
for i in 1:len
    if (value(x[i]) > 0)
        println(courses[i])
        opt_gpa += gpa[i]
    end
end
opt_gpa /= NUM_COURSES
println(opt_gpa)

COMP SCI 252
COMP SCI 300
COMP SCI 354
COMP SCI 400
COMP SCI 407
COMP SCI 412
COMP SCI 536
COMP SCI 545
COMP SCI 547
COMP SCI 558
COMP SCI 559
COMP SCI 577
COMP SCI 579
COMP SCI 679
MATH 234
MATH 461
MATH 221
MATH 222
MATH 240
STAT 324
3.2692855292


In [19]:
# Suppose a student have following preferences for classes to take:
# prefer CS577 to CS520 for degree requirement
# interested in optimization CS524 & CS525
# interested in database CS564
# interested in artificial intelligence CS540
# Except these preferences, the student just takes whatever other courses that give highest overall GPA.
preference = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,
              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

73-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [23]:
using JuMP, Cbc

NUM_COURSES = 15

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

@variable(m, x[1:len], Bin)  # whether or not a course is taken

# Must satisfy degree requirements
@constraint(m, sum( x[add_math[i]] for i in 1:length(add_math) ) >= 2)
@constraint(m, sum( x[theory[i]] for i in 1:length(theory) ) >= 1)
@constraint(m, sum( x[soft_hard[i]] for i in 1:length(soft_hard) ) >= 2)
@constraint(m, sum( x[app[i]] for i in 1:length(app) ) >= 1)
@constraint(m, sum( x[elect[i]] for i in 1:length(elect) ) >= 2)

# A class can not count for more than one degree requirement
@constraint(m, sum( x[i] for i in 1:len ) == NUM_COURSES)

# Must satisfy prerequisite requirements
for i in 1:len
    for j in pre[i]
        @constraint(m, x[i] <= x[j])
    end
end

# Meet student's preference
for i in 1:len
    @constraint(m, x[i] >= preference[i])
end

# Maximize GPA
@objective(m, Max, sum(x[i]*gpa[i] for i in 1:len) / NUM_COURSES);

optimize!(m)

opt_gpa = 0
for i in 1:len
    if (value(x[i]) > 0)
        println(courses[i])
        opt_gpa += gpa[i]
    end
end
opt_gpa /= NUM_COURSES
println(opt_gpa)

COMP SCI 252
COMP SCI 300
COMP SCI 354
COMP SCI 400
COMP SCI 407
COMP SCI 524
COMP SCI 525
COMP SCI 540
COMP SCI 564
COMP SCI 577
COMP SCI 579
MATH 320
MATH 221
MATH 222
MATH 240
3.180534099


In [22]:
gpa[17] #CS 524

3.306451613

In [24]:
gpa[18] #CS 525

2.949211909